Klasse regex_localizer

Findet Patterns in einer Anleitung anhand von regulären Ausdrücken.

Funktionen: 

    init( pattern_expressions)
        pattern_expressions:  Dictionary mit Key: Pattern ID; Value: Liste mit regulären Ausdrücken zu dem Pattern. 
        
    set_min_sentence_length(min_length)
        min_length: Mindest-Wortanzahl der Sätze. Sätze die weniger Wörter beinhalten, werden nicht geprüft.
    
    read_nlp_processed_game_to_game_patterns(nlp_processed_game)
        nlp_processed_game:      nlp_processed_game Objekt
        returns:                 game_patterns Objekt

In [1]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json
import re

In [2]:
%run ./GamePatterns.ipynb
%run ./nlpProcessedGame.ipynb

In [ ]:
class regex_localizer:
    #def __init__ (self, *args, **kwargs):
        
        
    def init(self, pattern_expressions, use_lemma):
        self.pattern_expressions = pattern_expressions
        self.min_length = 0
        self.use_lemma = use_lemma
            
    
            
    def set_min_sentence_length(self, min_length):
        self.min_length = min_length
        
    def read_nlp_processed_game_to_game_patterns(self,nlp_processed_game):
        gp = game_patterns()
        gp.set_ID(nlp_processed_game.get_ID())
        doc=nlp_processed_game.get_spacy_doc()
        #Alle Satzanfänge ermitteln
        sentences = {}
        for key, value in self.pattern_expressions.items():
                sentences[str(key)] = []
        sentenceTexts = []
        currSent = -1
        for sent in doc.sents:
            currSent += 1
            sentenceTexts.append(str(sent))
            if (len(sent)<self.min_length):
                continue
            for key, value in self.pattern_expressions.items():
                for expression in value:
                    match = ""
                    if(self.use_lemma):
                        match = re.search(expression, str(sent.lemma_), re.IGNORECASE)
                    else:
                        match = re.search(expression, str(sent), re.IGNORECASE)
                    if(match):
                        sentences[str(key)].append(currSent)
                        break
               
            
        gp.set_sentences(sentenceTexts)
        #print(sentences)
        for key, value in sentences.items():
            if(len(value)>0):
                gp.add_pattern(int(key),value.copy())
        
        return gp
    

test_patterns = {}
#test_patterns["73230"]=("jeden", "alle spieler", "jeder", "je", "die biber", "ihre startfelder", "auf die ausgangsfelder", "stellt sie", "eine spielfigur", "jedes", "alle ritter dieser farbe", "jeder eine tierart", "zahlenm\u00e4\u00dfig etwa gleichstarke", "das startfeld", "gestellt", "bringt eure boote", "jede/r", "den start", "alle", "startfeld", "eure figur", "zentrale")
test_patterns["73230"]=("jede.*start")